In [1]:
import numpy as np
import pandas as pd

In [2]:
jogos = pd.read_csv('matches_FLA2023.csv')
gols = pd.read_csv('goal_scorers.csv')

In [3]:
jogos

,id_match,home_team,away_team,home_score,away_score,tournament,kick-off,stadium
0,CA1,Flamengo,Audax-RJ,1,0,Carioca - Taça Guanabara - Round 5,23-01-12 21:30:00,Maracanã-RJ
1,CA2,Flamengo,Portuguesa-RJ,4,1,Carioca - Taça Guanabara - Round 1,23-01-15 18:00:00,Maracanã-RJ
2,CA3,Madureira-RJ,Flamengo,0,0,Carioca - Taça Guanabara - Round 2,23-01-18 19:00:00,Kléber Andrade-ES
3,CA4,Flamengo,Nova Iguaçu-RJ,5,0,Carioca - Taça Guanabara - Round 3,23-01-21 16:00:00,Maracanã-RJ
4,CA5,Bangu-RJ,Flamengo,1,1,Carioca - Taça Guanabara - Round 4,23-01-24 21:10:00,Raulino de Oliveira-RJ
5,SC1,Palmeiras,Flamengo,4,3,Supercopa do Brasil,23-01-28 16:30:00,Mané Garrincha-DF
6,CA6,Flamengo,Boavista,1,0,Carioca - Taça Guanabara - Round 6,23-02-01 21:10:00,Maracanã-RJ
7,CW1,Flamengo,Al-Hilal Saudi,2,3,Club World Championship - Semifinal,23-02-07 16:00:00,Ibn Batouta Stadium-Marrocos
8,CW2,Al Ahly SC,Flamengo,2,4,Club World Championship - 3rd place final,23-02-11 12:30:00,Complexe Sportif Moulay Abdellah-Marrocos


In [4]:
gols.head()

,id_match,gols,assist
0,CA1,Matheus França,Thiaguinho
1,CA2,Pedro,Ayrton Lucas
2,CA2,Gabi,Pedro
3,CA2,Fabrício Bruno,Pedro
4,CA2,Thiago Maia,Matheuzinho


In [5]:
df = pd.merge(jogos, gols, on = 'id_match')
df

,id_match,home_team,away_team,home_score,away_score,tournament,kick-off,stadium,gols,assist
0,CA1,Flamengo,Audax-RJ,1,0,Carioca - Taça Guanabara - Round 5,23-01-12 21:30:00,Maracanã-RJ,Matheus França,Thiaguinho
1,CA2,Flamengo,Portuguesa-RJ,4,1,Carioca - Taça Guanabara - Round 1,23-01-15 18:00:00,Maracanã-RJ,Pedro,Ayrton Lucas
2,CA2,Flamengo,Portuguesa-RJ,4,1,Carioca - Taça Guanabara - Round 1,23-01-15 18:00:00,Maracanã-RJ,Gabi,Pedro
3,CA2,Flamengo,Portuguesa-RJ,4,1,Carioca - Taça Guanabara - Round 1,23-01-15 18:00:00,Maracanã-RJ,Fabrício Bruno,Pedro
4,CA2,Flamengo,Portuguesa-RJ,4,1,Carioca - Taça Guanabara - Round 1,23-01-15 18:00:00,Maracanã-RJ,Thiago Maia,Matheuzinho
5,CA4,Flamengo,Nova Iguaçu-RJ,5,0,Carioca - Taça Guanabara - Round 3,23-01-21 16:00:00,Maracanã-RJ,Pedro,Matheuzinho
6,CA4,Flamengo,Nova Iguaçu-RJ,5,0,Carioca - Taça Guanabara - Round 3,23-01-21 16:00:00,Maracanã-RJ,Fabrício Bruno,Everton
7,CA4,Flamengo,Nova Iguaçu-RJ,5,0,Carioca - Taça Guanabara - Round 3,23-01-21 16:00:00,Maracanã-RJ,Gabi,Matheuzinho
8,CA4,Flamengo,Nova Iguaçu-RJ,5,0,Carioca - Taça Guanabara - Round 3,23-01-21 16:00:00,Maracanã-RJ,Gabi,Filipe Luis
9,CA4,Flamengo,Nova Iguaçu-RJ,5,0,Carioca - Taça Guanabara - Round 3,23-01-21 16:00:00,Maracanã-RJ,Pedro,Arrascaeta


In [6]:
df.gols.value_counts()

Pedro             9
Gabi              7
Fabrício Bruno    2
Matheus França    1
Thiago Maia       1
Lorran            1
Name: gols, dtype: int64

In [7]:
df.assist.value_counts()

Matheuzinho        4
Pedro              2
Thiaguinho         1
Ayrton Lucas       1
Everton            1
Filipe Luis        1
Arrascaeta         1
Mateusão           1
Everton Ribeiro    1
Gabi               1
Name: assist, dtype: int64

In [8]:
vitorias_casa = jogos[(jogos['home_team'] == 'Flamengo') & (jogos['home_score'] > jogos['away_score'])]
derrotas_casa = jogos[(jogos['home_team'] == 'Flamengo') & (jogos['home_score'] < jogos['away_score'])]

vitorias_fora = jogos[(jogos['away_team'] == 'Flamengo') & (jogos['away_score'] > jogos['home_score'])]
derrota_fora = jogos[(jogos['away_team'] == 'Flamengo') & (jogos['away_score'] < jogos['home_score'])]

empate_casa = jogos[(jogos['home_team'] == 'Flamengo') & (jogos['away_score'] == jogos['home_score'])]
empate_fora = jogos[(jogos['away_team'] == 'Flamengo') & (jogos['away_score'] == jogos['home_score'])]

In [9]:
vitorias_casa.shape[0] + vitorias_fora.shape[0]

5

In [10]:
derrotas_casa.shape[0] + derrota_fora.shape[0]

2

In [11]:
empate_casa.shape[0] + empate_fora.shape[0]

2